# PyTerrier Tutorial Walk Through

## 1.PyTerrier Indexing

In [1]:
%pip install -q python-terrier
import pyterrier as pt

Note: you may need to restart the kernel to use updated packages.


这里vaswani是一个很小的IR测试语料

print会显示该集合下所有文档文件的路径列表

In [2]:
dataset = pt.get_dataset("vaswani")

print(f"Files in vaswani corpus:{dataset.get_corpus()}")

Files in vaswani corpus:['C:\\Users\\50327/.pyterrier\\corpora\\vaswani\\corpus\\doc-text.trec']


指定存放目录，将来生成的索引会被存放到当前工作目录中的index文件夹下

在重新建索引之前，先删除可能已经存在的index文件夹，确保每次开始时的状态是干净的

In [3]:
index_path = "./index"

In [4]:
!rm -rf ./index
indexer = pt.TRECCollectionIndexer(index_path, blocks=True)

'rm' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


Exception: Unable to find JAVA_HOME

创建一个TREC格式集合的索引器

TREC格式知识点补充：

```xml
<DOC>
  <DOCNO>唯一文档编号</DOCNO>
  <DOCHDR>
    （可选，通常是文档的元信息，比如 URL、时间戳等）
  </DOCHDR>
  <HEADLINE>
    （可选，文档标题）
  </HEADLINE>
  <TEXT>
    文档正文内容……
  </TEXT>
</DOC>
```
`<DOC>` … `</DOC>`：一篇文档的开始与结束标志。

`<DOCNO>` … `</DOCNO>`：文档的唯一标识符（编号），检索系统用它来区分不同文档。

`<DOCHDR>` … `</DOCHDR>`：可选域，通常包含比如原始 URL、抓取时间、来源等头信息。

`<HEADLINE>` … `</HEADLINE>`：可选域，表示文章标题或摘要。

`<TEXT>` … `</TEXT>`：必选域，文档的主要文本内容，用于建立索引和检索。

这种格式的优点是结构清晰，便于自动化解析：

索引器能快速定位 `<TEXT>` 块，把其中的词项提取并倒排索引；

保留 `<DOCNO>` 方便查询结果中返回原始 ID；

可选的头信息和标题域能够支持更丰富的检索功能（比如对标题加权）。
在 PyTerrier 的 TRECCollectionIndexer 中，它会自动识别这些标签，将正文分词、记录词项位置（如果 blocks=True），并输出标准的倒排索引文件，之后就能用各种检索模型（BM25、QL、短语查询、距离模型等）来运行实验了。

+ index函数会把传入的文件列表（这里是 dataset.get_corpus() 返回的所有 TREC 文档文件）一篇篇读进来，分词、建立倒排表，并把索引写到之前 index_path 指定的 ./index 目录下。

+ 返回的 indexref 是一个 Python 层面的“引用”对象，指向索引生成的元数据文件（Terrier 内部叫 data.properties），相当于一个 URI 或指针。

In [ ]:
indexref = indexer.index(dataset.get_corpus())

In [ ]:
indexref.toString()

'./index/data.properties'

In [ ]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 0
Number of tokens: 271581
Field names: []
Positions:   true



为一个pandas的dataframe创建索引

In [ ]:
import pandas as pd
!rm -rf ./pd_index
pd_indexer = pt.IterDictIndexer("./pd_index") #默认不记录词的位置
# pd_indexer = pt.IterDictIndexer("./pd_index", blocks=True)
#小规模或原型验证时，使用字典迭代索引器（接受一个可迭代的字典列表）更加方便，如果规模较大且有现成的TREC文件，则用TRECCollectionIndexer

In [ ]:
df = pd.DataFrame({
    'docno': ['1', '2', '3'],
    'url': ['url1','url2','url3'],
    'text': [
        'He ran out of money, so he had to stop playing',
        'The waves were crashing on the shore; it was a',
        'The body may perhaps compensates for the loss'
    ]
})
df

,docno,url,text
0,1,url1,"He ran out of money, so he had to stop playing"
1,2,url2,The waves were crashing on the shore; it was a
2,3,url3,The body may perhaps compensates for the loss


In [ ]:
# 不记录元数据
# pd_indexer.index(df["text"])

# 记录元数据,records是告诉index函数将每一行都当成一个记录来输出成一个列表
indexref2 = pd_indexer.index(df.to_dict(orient='records'))
# pd_indexer.index(df["text"], df["docno"], df["url"]) 这里正文必须是第一个Series

Indexing a iterable/generator
直接将索引器传入迭代器中，索引器

**生成器流式产出**

antique_doc_iter() 每次 yield 一个像 {'docno': …, 'text': …} 的字典，代表一篇文档。

**传入索引器**

把这个生成器对象直接传给
`iter_indexer.index(doc_iter)`
索引器会在内部循环调用 next(doc_iter)，拿到每个字典。



**边读边索引**

对每个字典：

从 text 字段分词并更新倒排结构（写到磁盘上的索引文件里）

把 docno（和其它元字段，如果有的话）写入文档映射表



**常数级内存开销**

因为文档是一条条来，不需要事先把所有文档读入一个列表或 DataFrame，用户侧（Python 进程）只需保留一个字典/一行的开销。索引器自己会按块（block）或批次把中间结构写到磁盘，内存使用不会随着文档数量线性增长。

In [ ]:
import urllib
import io
def antique_doc_iter():
  stream = urllib.request.urlopen('https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt')
  stream = io.TextIOWrapper(stream)
  for i, line in enumerate(stream):
    if i % 100000 == 0:
      print(f'processing document {i}')
    docno, text = line.rstrip().split('\t')
    yield {'docno': docno, 'text': text}

!rm -rf ./iter_index
# 不带元数据的索引
iter_indexer = pt.IterDictIndexer("./iter_index")

# 只有传入的参数实现了迭代接口，索引器才会主动调用next函数
doc_iter = antique_doc_iter()
indexref3 = iter_indexer.index(doc_iter)

processing document 0
16:24:28.694 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (730691_1) - further warnings are suppressed
processing document 100000
processing document 200000
processing document 300000
processing document 400000
16:25:21.715 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 2224 empty documents


Retrieval

In [ ]:
pt.terrier.Retriever(indexref).search("mathematical")

,qid,docid,docno,rank,score,query
0,1,303,304,0,3.566201,mathematical
1,1,2444,2445,1,3.566201,mathematical
2,1,3534,3535,2,3.566201,mathematical
3,1,5040,5041,3,3.566201,mathematical
4,1,1169,1170,4,3.564534,mathematical
...,...,...,...,...,...,...
147,1,7283,7284,147,2.834784,mathematical
148,1,6714,6715,148,2.811375,mathematical
149,1,4746,4747,149,2.790373,mathematical
150,1,8622,8623,150,2.759409,mathematical


In [ ]:
import pandas as pd
topics = pd.DataFrame([["2", "mathematical"],
            ["3", "chemical"]], columns=['qid', 'query'])
pt.terrier.Retriever(indexref).transform(topics)

,qid,docid,docno,rank,score,query
0,2,303,304,0,3.566201,mathematical
1,2,2444,2445,1,3.566201,mathematical
2,2,3534,3535,2,3.566201,mathematical
3,2,5040,5041,3,3.566201,mathematical
4,2,1169,1170,4,3.564534,mathematical
...,...,...,...,...,...,...
167,3,6128,6129,15,4.488467,chemical
168,3,4053,4054,16,4.211867,chemical
169,3,8415,8416,17,4.162157,chemical
170,3,3319,3320,18,4.018362,chemical


[experiment]To peep what vaswani looks like

In [ ]:
from xml.etree import ElementTree as ET

dataset = pt.get_dataset("vaswani")
corpus_files = dataset.get_corpus()[0]
print("Vaswani Corpus Files", corpus_files)

with open(corpus_files, "r", encoding='utf-8') as f:
  for i, line in enumerate(f):
    if i >= 10:
      break
    print(line.rstrip())

Vaswani Corpus Files /root/.pyterrier/corpora/vaswani/corpus/doc-text.trec
<DOC>
<DOCNO>1</DOCNO>
compact memories have flexible capacities  a digital data storage
system with capacity up to bits and random and or sequential access
is described
</DOC>
<DOC>
<DOCNO>2</DOCNO>
an electronic analogue computer for solving systems of linear equations
mathematical derivation of the operating principle and stability


## 2.Retrieval and Evaluation

Preparation & Load an existing index

In [ ]:
!pip install -q python-terrier
import pyterrier as pt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")

In [ ]:
# Here is just to have a review of how the indexref is built, but the fact is,
# there is already an indexref built inside the dataset to be called directly. See the block below.
# index_path = "./index"
# !rm -rf ./index
# indexer = pt.TRECCollectionIndexer(index_path, blocks=True)
# indexref = indexer.index(vaswani_dataset.get_corpus())

In [ ]:
indexref = vaswani_dataset.get_index()
index = pt.IndexFactory.of(indexref)

print(index.getCollectionStatistics().toString())

data.direct.bf:   0%|          | 0.00/388k [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/234k [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/362k [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/682k [00:00<?, ?iB/s]

data.lexicon.fsomaphash:   0%|          | 0.00/777 [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/89.3k [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/224k [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.29k [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/619 [00:00<?, ?iB/s]

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 1
Number of tokens: 271581
Field names: [text]
Positions:   false



Retrieval

In [ ]:
# 通常我们会使用这种方法来解析topics文件
# topics_path = "./query-text.trec"
# topics = pt.io.read_topics(topics_path)

In [ ]:
# pt.dataset提供的方法
topics = vaswani_dataset.get_topics()
topics.head(5)

query-text.trec:   0%|          | 0.00/3.05k [00:00<?, ?iB/s]

,qid,query
0,1,measurement of dielectric constant of liquids ...
1,2,mathematical analysis and design details of wa...
2,3,use of digital computers in the design of band...
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of comp...


创建Retriever对象

In [ ]:
# 这三种方法有一种就足够了
retr = pt.terrier.Retriever(index, controls={"wmodel": "TF_IDF"})

# retr.setControl("wmodel", "TF_IDF")
# retr.setControls({"wmodel": "TF_IDF"})

res = retr.transform(topics)

+ index:索引实例
+ controls:一个字典，用来传递给Terrier内部的控制参数
  + 这里 wmodel 就是 weighting model（权重模型）的简称，常见可选值有：

    + "BM25"

    + "TF_IDF"

    + "DirichletLM"

    + …

如果不显式传 controls，默认会用 Terrier 配置文件里的默认检索模型（通常是 BM25）。


In [ ]:
res

,qid,docid,docno,rank,score,query
0,1,8171,8172,0,13.746087,measurement of dielectric constant of liquids ...
1,1,9880,9881,1,12.352666,measurement of dielectric constant of liquids ...
2,1,5501,5502,2,12.178153,measurement of dielectric constant of liquids ...
3,1,1501,1502,3,10.993585,measurement of dielectric constant of liquids ...
4,1,9858,9859,4,10.271452,measurement of dielectric constant of liquids ...
...,...,...,...,...,...,...
91925,93,2226,2227,995,4.904950,high frequency oscillators using transistors t...
91926,93,6898,6899,996,4.899385,high frequency oscillators using transistors t...
91927,93,3473,3474,997,4.898796,high frequency oscillators using transistors t...
91928,93,3187,3188,998,4.893073,high frequency oscillators using transistors t...


In [ ]:
retr.search("Light")

,qid,docid,docno,rank,score,query
0,1,10808,10809,0,5.537595,Light
1,1,11231,11232,1,5.535640,Light
2,1,11066,11067,2,5.497895,Light
3,1,5995,5996,3,5.486707,Light
4,1,4460,4461,4,5.464468,Light
...,...,...,...,...,...,...
120,1,4820,4821,120,1.964441,Light
121,1,9836,9837,121,1.927833,Light
122,1,7213,7214,122,1.910036,Light
123,1,6177,6178,123,1.892565,Light


在查询单个字符串时，查询器内部会构造一个临时的、只有一行的DataFrame

In [ ]:
pt.io.write_results(res, "result1.res")

Evaluation

In [ ]:
# # 使用本地测试集合
# qrels_path = ("./qrels")
# qrels = pt.io.read_qrels(qrels_path)

In [ ]:
# 使用Vaswani数据集中现成的qrels
qrels = vaswani_dataset.get_qrels()

In [ ]:
eval = pt.Evaluate(res, qrels)
eval

{'map': 0.29090543005529873, 'ndcg': 0.6153667539666847}

MAP（Mean Average Precision 平均平均精度）

**定义**  
对每个查询计算 Average Precision (AP)，再对所有查询取平均。

1. **单查询的 AP**  
   对于查询 $q$，假设检索结果排序后共有 $N$ 个文档，第 $k$ 位的文档相关性标签为 $\mathrm{rel}_k \in \{0,1\}$，该查询共有 $R$ 个相关文档，则  
   $$
   \mathrm{AP}_q
     = \frac{1}{R}
       \sum_{k=1}^{N}
         P(k)\,\mathbb{1}[\mathrm{rel}_k = 1]
   $$  
   
   $\mathbb{1}[\cdot]$ 为指示函数，当文档第 $k$ 位相关时取 1，否则取 0。

2. **所有查询的 MAP**  
   对所有 $Q$ 个查询的 AP 取平均：  
   $$
   \mathrm{MAP}
     = \frac{1}{Q}
       \sum_{q=1}^{Q}
         \mathrm{AP}_q
   $$

> **取值范围**：$[0,1]$。  
> 在你的实验中，MAP≈0.2909 表示系统平均把相关文档排到前面的精度约为 29.1%。

---

nDCG（Normalized Discounted Cumulative Gain 归一化折损累积增益）

**定义**  
在排序列表前 $K$ 个位置累积增益，并作位置折损，再归一化。

1. **DCG@K**  
   $$
   \mathrm{DCG}@K
     = \sum_{i=1}^K
         \frac{2^{\mathrm{rel}_i} - 1}{\log_2(i + 1)}
   $$  
   其中 $\mathrm{rel}_i$ 是排名第 $i$ 位文档的相关性等级（二元下为 0 或 1，多级相关时可为更大整数）。

2. **理想 DCG (IDCG@K)**  
   将所有相关文档按最大相关度排序后得到的 DCG@K，表示“最佳可能得分”。

3. **nDCG@K**  
   $$
   \mathrm{nDCG}@K
     = \frac{\mathrm{DCG}@K}{\mathrm{IDCG}@K}
   $$

> **取值范围**：$[0,1]$。  
> 在你的实验中，nDCG≈0.6154 表明相对于理想排序，你的系统达到了约 61.5% 的效果。

---

小结

- **MAP** 偏向衡量“相关文档平均精度”，对所有相关文档的检索排名都敏感。  
- **nDCG** 强调“高相关文档排在前面”，并且天然支持多级相关度标签。  

两者结合使用，可以从全局精度和前端排序质量两个角度评价检索系统的表现。


Another Attempt Using BM25

In [ ]:
retr2 = pt.terrier.Retriever(index, controls={"wmodel": "BM25"})
res2 = retr2.transform(topics)

In [ ]:
eval2 = pt.Evaluate(res2, qrels, metrics=['map'])
eval2

{'map': 0.296517205483994}

## 3.Tutorial Part1 Classical IR


